In [1]:
import requests

***
***

In [2]:
class UCloudRequester:

    def __init__(self, url : str, email : str) -> None:
        self.url = url
        self.email = email

    def _predict(self, prompt : str, **params) -> dict:

        try:
            query = {
                'email': self.email,
                'prompt': prompt,
                'params': {**params}
            }
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.post(self.url, headers=headers, json=query)
            return response.json()
        
        except requests.exceptions.JSONDecodeError:
            return f"Failed to decode response as JSON: {response.text}"

        except requests.exceptions.HTTPError as e:
            return "Error: " + str(e)
        
    
    def predict(self, prompt : str, return_meta=False, **params) -> dict | str:
        result : dict = self._predict(prompt, **params)
        if return_meta:
            return result
        return result["choices"][0]["text"]
    
requester = UCloudRequester(
    url="https://app-aiml24.cloud.sdu.dk/predict",
    email="nbth.msc@cbs.dk"
)


***
***

In [10]:
params = dict(
    max_tokens=1000,
    temperature=0.5,
    stop=["\n", "###"],
    # logprobs=2,
)

In [11]:
PROMPT = """Malaysia Airlines Flight 370 (MH370/MAS370)[a] was an international passenger flight operated by Malaysia Airlines that disappeared from radar on 8 March 2014 while flying from Kuala Lumpur International Airport in Malaysia to its planned destination, Beijing Capital International Airport in China.[1] The reason for its disappearance has not been determined.
The crew of the Boeing 777-200ER, registered as 9M-MRO, last communicated with air traffic control (ATC) around 38 minutes after takeoff when the flight was over the South China Sea. The aircraft was lost from ATC's secondary surveillance radar screens minutes later, but was tracked by the Malaysian military's primary radar system for another hour, deviating westward from its planned flight path, crossing the Malay Peninsula and Andaman Sea. It left radar range 200 nautical miles (370 km; 230 mi) northwest of Penang Island in northwestern Peninsular Malaysia.
With all 227 passengers and 12 crew aboard presumed dead, the disappearance of Flight 370 was the deadliest incident involving a Boeing 777 and the deadliest in Malaysia Airlines' history until it was surpassed in both regards by Malaysia Airlines Flight 17, which was shot down while flying over Ukraine four months later on 17 July 2014. The combined loss caused significant financial problems for Malaysia Airlines, which was renationalised by the Malaysian government in August 2014.
The search for the missing airplane became the most expensive search in the history of aviation. It focused initially on the South China Sea and Andaman Sea, before a novel analysis of the aircraft's automated communications with an Inmarsat satellite indicated that the plane had traveled far southward over the southern Indian Ocean. The lack of official information in the days immediately after the disappearance prompted fierce criticism from the Chinese public, particularly from relatives of the passengers, as most people on board Flight 370 were of Chinese origin. Several pieces of debris washed ashore in the western Indian Ocean during 2015 and 2016; many of these were confirmed to have originated from Flight 370. What is the name the company that owned Flight 370? 
"""

In [12]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [13]:
prediction

{'id': 'cmpl-389114f6-41d1-42b0-a963-8260d010fbf2',
 'object': 'text_completion',
 'created': 1710204494,
 'model': 'models/mixtral-8x7b-instruct-v0.1.Q6_K.gguf',
 'choices': [{'text': 'Malaysia Airlines',
   'index': 0,
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 516, 'completion_tokens': 4, 'total_tokens': 520},
 'timestamp': '2024-03-12T00:48:14.357914',
 'email': 'nbth.msc@cbs.dk'}

In [ ]:
PROMPT = """Write a poem about the meaning of life.
Poem:

"""

In [ ]:
prediction = requester.predict(prompt=PROMPT, return_meta=True, **params)

In [67]:
%%timeit -n 10 -r 10

requester.predict(prompt=PROMPT, **params)

769 ms ± 47.1 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [68]:
%%timeit -n 10 -r 10

requester.predict(f"The meaning of life is", **params)

741 ms ± 134 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
